In [95]:
# Import packages
import xarray as xr
import numpy as np
import subprocess
import pandas as pd
import glob
import os
import sys
import ddeq
from datetime import datetime as dt

In [307]:
def vertical_interp_icon(ds_icon, ds_interpolated, ds_camchem, species_list=['NO2', 'NO', 'HNO3', 'O3', 'H2NO5', 'OH', 'CH3CHO']):

    for species in species_list:

        ###########################################################################
        ###########################################################################
        ds_camchem['pres_ifc'] = ds_camchem['hyai']*ds_camchem['P0'] + ds_camchem['hybi']*ds_camchem['PS'] # Seperating levels
        ds_camchem['pres'] = ds_camchem['hyam']*ds_camchem['P0'] + ds_camchem['hybm']*ds_camchem['PS'] # level on which the variables are defined
        ds_camchem['pres_ifc'] = ds_camchem['pres_ifc'].transpose('time', 'ilev', 'ncells')
        ds_camchem['pres'] = ds_camchem['pres'].transpose('time', 'lev', 'ncells')

        # Perform interpolation for surface levels

        # Dimension order: level, ncells
        icon_depths = ds_icon['z_ifc'][:-1].values-ds_icon['z_ifc'][1:].values
        icon_pressures = ds_icon['pres_ifc'][0].values
        camchem_pressures = ds_camchem['pres_ifc'][0].values

        icon_surface_p = ds_icon['pres_ifc'][0,-1].values
        camchem_surface_p = ds_camchem['pres_ifc'][0,-1].values

        # Pressure difference within first camchem layer
        camchem_first_layer_p_diff =  camchem_surface_p - camchem_pressures[-2]

        # Find target pressure
        target_pressure = icon_surface_p - camchem_first_layer_p_diff

        # Finds the first index where model lies below target level (so model pressure is higher than target pressure)
        above_target = icon_pressures[:,np.newaxis] >= target_pressure
        first_negative_indices = np.argmax(above_target, axis=0)

        # Select indices of 2 closest neighbouring levels
        vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target

        # # Select pressure values of 2 closest neighbouring levels
        pk_low = np.take_along_axis((icon_pressures), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
        pk_up = np.take_along_axis((icon_pressures), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

        # Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
        alpha_low = (np.log(target_pressure/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
        alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

        # Find which icon levels fully lie within the camchem layer
        nr_full_layers = 60 - first_negative_indices

        # Calculate the maximum number of full_count_levels
        max_levels = np.max(nr_full_layers)

        # Initialize an empty list to store arrays
        full_count_levels_arrays = []
        weights_full_levels = []
        depth_weights = []

        # Loop through each level
        for i in range(0, max_levels+1):
            # Create a new array based on the condition
            new_array = np.where(nr_full_layers > i, vertical_indices[0] + i, 0)
            new_array2 = np.where(nr_full_layers > i, 1, 0)
            # Append the new array to the list
            full_count_levels_arrays.append(new_array)
            weights_full_levels.append(new_array2)

        # Select the species from the icon dataset
        # icon_subset = ds_icon[species + '_full'][0].values

        # Flip ICON species to match TM5 vertical levels and drop the time dimension
        # Check if the species is 'temp'
        if species == 'temp':
            icon_subset = ds_icon[species].values[0]
        else:
            icon_subset = ds_icon[species + '_full'].values[0]


        # Initialize ds_interp with zeros of the same shape as icon_subset to calculate the weighted average
        ds_interp_surface = np.zeros((1, icon_subset.shape[1]))

        # Initialize a count array to keep track of the number of non-zero contributing weights
        count_array = np.zeros((1, icon_subset.shape[1]))

        ds_interp_surface = np.take_along_axis(icon_subset, vertical_indices[1], axis=0) * alpha_complete[1]

        store_arrays = []
        store_counts = []
        store_depths = []

        # Loop through each level array and its corresponding weight array
        for i in range(0, max_levels):
            # Get the weighted interpolated array for the current level
            weighted_interpolated_array = np.take_along_axis(icon_subset, full_count_levels_arrays[i], axis=0) * weights_full_levels[i]
            depths_array = np.take_along_axis(icon_depths, full_count_levels_arrays[i], axis=0) * weights_full_levels[i]

            # Update ds_interp with the weighted interpolated values
            # ds_interp_surface += weighted_interpolated_array
            
            # Update count_array by adding 1 where weights are non-zero
            count_array += (weights_full_levels[i] != 0)

            store_arrays.append(weighted_interpolated_array)
            store_counts.append(weights_full_levels[i])
            store_depths.append(depths_array)

        # # Calculate the average by dividing by the count_array
        # ds_interp_surface /= count_array + alpha_complete[1]

        # Multiply store_arrays by store_depths element-wise
        weighted_arrays = np.array(store_arrays) * np.array(store_depths) # weigh concentration by depth
        # Sum along the first axis to get a single array
        summed_array = np.sum(weighted_arrays, axis=0) # sum over all full layers

        # Weigh the arrat 
        half_layer_depth_array = np.take_along_axis(icon_depths, full_count_levels_arrays[0]-1, axis=0) # depth of the icon layer that party falls within the camchem layer
        weighted_array_half_levels = ds_interp_surface * half_layer_depth_array*alpha_complete[0] # weigh concentration by depth

        ds_interp_surface =  summed_array + weighted_array_half_levels

        ds_interp_surface /= np.sum(store_depths, axis=0) + half_layer_depth_array*alpha_complete[0]

        ###########################################################################
        # Perform interpolation for other levels

        # Interpolate the ICON vertical profile
        ICON_levels = np.flip(ds_icon.pres[0,:,:].values,axis=0)
        target_pressures = ds_new['pres'].values

        # Finds the first index where model lies below target level (so model pressure is higher than target pressure)
        # this function gives a 0 when the sample lies below the lowest model level, and a -1 if the sample lies above the highest model level                   
        above_target = ICON_levels[:,np.newaxis] <= target_pressures
        lev_count=(ICON_levels[:,np.newaxis]>=target_pressures).sum(axis=0)
        first_negative_indices=np.where(lev_count==len(ICON_levels),-1,np.argmax(above_target,axis=0)) 

        # Select indices of 2 closest neighbouring levels
        vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target
        vertical_indices[:,first_negative_indices==0] = 0 # If the sample lies below the lowest model level. Set it to the lowest model level !!Note: this also asigns a 0 if the pressure is nan, but for our case, these points are filtered out later anyways
        vertical_indices[:,first_negative_indices==-1] = len(ICON_levels)-1 # If the sample lies above the highest model level. Set it to the highest model level

        # Select pressure values of 2 closest neighbouring levels
        pk_low = np.take_along_axis((ICON_levels), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
        pk_up = np.take_along_axis((ICON_levels), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

        # Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
        alpha_low = (np.log(target_pressures/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
        alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

        # Correct alpha for levels lower or higher than the model levels
        alpha_complete[:,first_negative_indices==0] = 0.5 # If sample lies below lowest model level, give all weight to the lowest model level
        alpha_complete[:,first_negative_indices==-1] = 0.5 # If sample lies above the highest model level, give all weight to the highest model level

        # Flip ICON species to match TM5 vertical levels and drop the time dimension
        # Check if the species is 'temp'
        if species == 'temp':
            ICON_subset = np.flip(ds_icon[species].values[0], axis=0)
        else:
            ICON_subset = np.flip(ds_icon[species + '_full'].values[0], axis=0)

        # Perform the interpolation using the determined weights
        species_interp = np.take_along_axis(ICON_subset, vertical_indices[1], axis=0) * alpha_complete[0] + \
                        np.take_along_axis(ICON_subset, vertical_indices[0], axis=0) * alpha_complete[1]

        # Add the interpolated ICON species to the TROPOMI dataset
        ds_interpolated[species] = (('pres', 'ncells'), species_interp)

        ds_interpolated[species][0] = ds_interp_surface[0]

    return ds_interpolated


In [322]:
#### OLD CODE

# def vertical_interp_icon(ds_icon, ds_interpolated, ds_camchem, species_list=['NO2', 'NO', 'HNO3', 'O3', 'H2NO5', 'OH', 'CH3CHO']):

#     for species in species_list:

#         ###########################################################################
#         ds_camchem['pres_ifc'] = ds_camchem['hyai']*ds_camchem['P0'] + ds_camchem['hybi']*ds_camchem['PS'] # Seperating levels
#         ds_camchem['pres'] = ds_camchem['hyam']*ds_camchem['P0'] + ds_camchem['hybm']*ds_camchem['PS'] # level on which the variables are defined
#         ds_camchem['pres_ifc'] = ds_camchem['pres_ifc'].transpose('time', 'ilev', 'ncells')
#         ds_camchem['pres'] = ds_camchem['pres'].transpose('time', 'lev', 'ncells')
        
#         # Perform interpolation for surface levels

#         # Dimension order: level, ncells
#         icon_pressures = ds_icon['pres_ifc'][0].values
#         camchem_pressures = ds_camchem['pres_ifc'][0].values

#         icon_surface_p = ds_icon['pres_ifc'][0,-1].values
#         camchem_surface_p = ds_camchem['pres_ifc'][0,-1].values

#         # Pressure difference within first camchem layer
#         camchem_first_layer_p_diff =  camchem_surface_p - camchem_pressures[-2]

#         # Find target pressure
#         target_pressure = icon_surface_p - camchem_first_layer_p_diff

#         # Finds the first index where model lies below target level (so model pressure is higher than target pressure)
#         above_target = icon_pressures[:,np.newaxis] >= target_pressure
#         first_negative_indices = np.argmax(above_target, axis=0)

#         # Select indices of 2 closest neighbouring levels
#         vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target

#         # # Select pressure values of 2 closest neighbouring levels
#         pk_low = np.take_along_axis((icon_pressures), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
#         pk_up = np.take_along_axis((icon_pressures), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

#         # Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
#         alpha_low = (np.log(target_pressure/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
#         alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

#         # Find which icon levels fully lie within the camchem layer
#         nr_full_layers = 60 - first_negative_indices

#         # Calculate the maximum number of full_count_levels
#         max_levels = np.max(nr_full_layers)

#         # Initialize an empty list to store arrays
#         full_count_levels_arrays = []
#         weights_full_levels = []

#         # Loop through each level
#         for i in range(0, max_levels):
#             # Create a new array based on the condition
#             new_array = np.where(nr_full_layers > i, vertical_indices[0] + i, 0)
#             new_array2 = np.where(nr_full_layers > i, 1, 0)
#             # Append the new array to the list
#             full_count_levels_arrays.append(new_array)
#             weights_full_levels.append(new_array2)

#         # Select the species from the icon dataset
#         if species == 'temp':
#             icon_subset = np.flip(ds_icon[species].values[0], axis=0)
#         else:
#             icon_subset = np.flip(ds_icon[species + '_full'].values[0], axis=0)

#         # Initialize ds_interp with zeros of the same shape as icon_subset to calculate the weighted average
#         ds_interp_surface = np.zeros((1, icon_subset.shape[1]))

#         # Initialize a count array to keep track of the number of non-zero contributing weights
#         count_array = np.zeros((1, icon_subset.shape[1]))

#         ds_interp_surface = np.take_along_axis(icon_subset, vertical_indices[1], axis=0) * alpha_complete[1]

#         # Loop through each level array and its corresponding weight array
#         for i in range(0, max_levels):
#             # Get the weighted interpolated array for the current level
#             weighted_interpolated_array = np.take_along_axis(icon_subset, full_count_levels_arrays[i], axis=0) * weights_full_levels[i]
            
#             # Update ds_interp with the weighted interpolated values
#             ds_interp_surface += weighted_interpolated_array
            
#             # Update count_array by adding 1 where weights are non-zero
#             count_array += (weights_full_levels[i] != 0)

#         # Calculate the average by dividing by the count_array
#         ds_interp_surface /= count_array + alpha_complete[1]

#         ###########################################################################
#         # Perform interpolation for other levels

#         # Interpolate the ICON vertical profile
#         ICON_levels = np.flip(ds_icon.pres[0,:,:].values,axis=0)
#         target_pressures = ds_new['pres'].values

#         # Finds the first index where model lies below target level (so model pressure is higher than target pressure)
#         # this function gives a 0 when the sample lies below the lowest model level, and a -1 if the sample lies above the highest model level                   
#         above_target = ICON_levels[:,np.newaxis] <= target_pressures
#         lev_count=(ICON_levels[:,np.newaxis]>=target_pressures).sum(axis=0)
#         first_negative_indices=np.where(lev_count==len(ICON_levels),-1,np.argmax(above_target,axis=0)) 

#         # Select indices of 2 closest neighbouring levels
#         vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target
#         vertical_indices[:,first_negative_indices==0] = 0 # If the sample lies below the lowest model level. Set it to the lowest model level !!Note: this also asigns a 0 if the pressure is nan, but for our case, these points are filtered out later anyways
#         vertical_indices[:,first_negative_indices==-1] = len(ICON_levels)-1 # If the sample lies above the highest model level. Set it to the highest model level

#         # Select pressure values of 2 closest neighbouring levels
#         pk_low = np.take_along_axis((ICON_levels), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
#         pk_up = np.take_along_axis((ICON_levels), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

#         # Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
#         alpha_low = (np.log(target_pressures/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
#         alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

#         # Correct alpha for levels lower or higher than the model levels
#         alpha_complete[:,first_negative_indices==0] = 0.5 # If sample lies below lowest model level, give all weight to the lowest model level
#         alpha_complete[:,first_negative_indices==-1] = 0.5 # If sample lies above the highest model level, give all weight to the highest model level

#         # Flip ICON species to match TM5 vertical levels and drop the time dimension
#         # Check if the species is 'temp'
#         if species == 'temp':
#             ICON_subset = np.flip(ds_icon[species].values[0], axis=0)
#         else:
#             ICON_subset = np.flip(ds_icon[species + '_full'].values[0], axis=0)

#         # Perform the interpolation using the determined weights
#         species_interp = np.take_along_axis(ICON_subset, vertical_indices[1], axis=0) * alpha_complete[0] + \
#                         np.take_along_axis(ICON_subset, vertical_indices[0], axis=0) * alpha_complete[1]

#         # Add the interpolated ICON species to the TROPOMI dataset
#         ds_interpolated[species] = (('pres', 'ncells'), species_interp)

#         ds_interpolated[species][0] = ds_interp_surface[0]

#     return ds_interpolated


In [324]:

# Define the paths to the ICON files
base_path = '/scratch/snx3000/amols/data/nudged_ICON_data'
out_path = '/scratch/snx3000/amols/data/nudged_ICON_data/pres_interp'

# Define the list of months, days, and hours
# months = ['08']  # Add more months as needed
months = ['02', '07']  # Add more months as needed
days = list(range(12, 29))  # Days 13-26
# days = list(range(1, 12))  # Days 13-26
#days = list(range(29, 32))  # Days 13-26
# hours = ['00', '06', '12', '18']  # Add more hours as needed
hours = ['12', '00']  # Add more hours as needed

# Iterate over months, days, and hours
for month in months:
    for day in days:
        for hour in hours:
            # Load the ICON dataset for the specified month, day, and hour
            icon_file = os.path.join(base_path, f'icon-art-full-chem_EU_LBC_2019{month}{day:02d}T{hour}0000Z.nc')
            ds_icon = xr.open_dataset(icon_file)

            camchem_file = f'/scratch/snx3000/amols/data/camchem_data/camchem_2019{month}{day:02d}{hour}_icon.nc'
            ds_camchem = xr.open_dataset(camchem_file)

            # Extracting the selected variables from ds_icon
            selected_variables = ['clon', 'clat', 'clon_bnds', 'clat_bnds', 'vertices']

            # Creating a new dataset with selected variables
            ds_new = xr.Dataset({var: ds_icon[var] for var in selected_variables})

            # Creating a new DataArray for the 'pres' variable using target_pressures
            pres_values = np.array([100000, 95000, 90000, 80000, 60000, 40000])
            ncells_values = ds_icon['ncells'].values

            # Repeat the 'pres' values for each 'ncells'
            pres_dataarray = xr.DataArray(np.tile(pres_values, (len(ncells_values), 1)), dims=('ncells', 'pres'),
                                          coords={'ncells': ncells_values, 'pres': pres_values})

            # Add the value of pres_sfc to pres_dataarray
            pres_sfc_value = ds_icon['pres_sfc'][0].values
            
            # Concatenate pres_sfc_value with pres_dataarray along the second axis
            pres_values_with_sfc = np.concatenate((np.expand_dims(pres_sfc_value, axis=1), pres_dataarray), axis=1)

            # Update pres_dataarray with the concatenated values
            pres_dataarray = xr.DataArray(pres_values_with_sfc, dims=('ncells', 'pres'),
                                        coords={'ncells': ncells_values, 'pres': pres_values_with_sfc[0]})

            # Adding the 'pres' variable to the new dataset
            ds_new['pres'] = pres_dataarray

            # Transpose the 'pres' variable to reorder the coordinates
            ds_new['pres'] = ds_new['pres'].transpose('pres', 'ncells')

            # Perform vertical interpolation for the specified species
            species_list = ['NO2', 'NO', 'HNO3', 'O3', 'N2O5', 'OH', 'ALKNO3', 'HNO4', 'LHONITR', 'LISOPBDNO3',
                            'LISOPACNO3', 'MPAN', 'NO3', 'NOA', 'LONITR', 'PAN', 'PBZNIT', 'BPINANO3', 'CH3CHO', 'CO', 'temp']
            ds_interpolated = vertical_interp_icon(ds_icon, ds_new, ds_camchem, species_list)

            ds_interpolated['NOx'] = ds_interpolated['NO2'] + ds_interpolated['NO']
            ds_interpolated['NOz'] = ds_interpolated['HNO3'] + ds_interpolated['N2O5'] + ds_interpolated['ALKNO3'] + \
                                    ds_interpolated['HNO4'] + ds_interpolated['LHONITR'] + ds_interpolated['LISOPBDNO3'] + \
                                    ds_interpolated['LISOPACNO3'] + ds_interpolated['MPAN'] + ds_interpolated['NO3'] + \
                                    ds_interpolated['NOA'] + ds_interpolated['LONITR'] + ds_interpolated['PAN'] + \
                                    ds_interpolated['PBZNIT'] + ds_interpolated['BPINANO3']
            ds_interpolated['NOy'] = ds_interpolated['NOx'] + ds_interpolated['NOz']
            ds_interpolated['NOx_NOy'] = ds_interpolated['NOx']/ds_interpolated['NOy']
            ds_interpolated['NOx_NO2'] = ds_interpolated['NOx']/ds_interpolated['NO2']


            # Create a directory to store the interpolated datasets if it doesn't exist
            if not os.path.exists(out_path):
                os.makedirs(out_path)

            # Define the output filename
            out_filename = f'interp_icon_2019{month}{day:02d}T{hour}.nc'
            out_file = os.path.join(out_path, out_filename)

            if os.path.exists(out_file):
                # Delete the file
                os.remove(out_file)

            # Save the interpolated dataset to a NetCDF file
            ds_interpolated.to_netcdf(out_file)

            print(f'Interpolated dataset saved: {out_file}')


Interpolated dataset saved: /scratch/snx3000/amols/data/nudged_ICON_data/pres_interp/interp_icon_20190212T12.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/nudged_ICON_data/pres_interp/interp_icon_20190212T00.nc


#### Interpolation for camchem

In [319]:
def vertical_interp_camchem(ds_camchem, ds_interpolated, species_list=['NO2', 'NO', 'HNO3', 'O3', 'H2NO5', 'OH', 'CH3CHO']):

    for species in species_list:
        # Interpolate the ICON vertical profile
        camchem_levels = np.flip(ds_camchem.pres[:,0,:].values,axis=0)
        target_pressures = ds_interpolated['pres'].values

        # Finds the first index where model lies below target level (so model pressure is higher than target pressure)
        # this function gives a 0 when the sample lies below the lowest model level, and a -1 if the sample lies above the highest model level                   
        above_target = camchem_levels[:,np.newaxis] <= target_pressures
        lev_count=(camchem_levels[:,np.newaxis]>=target_pressures).sum(axis=0)
        first_negative_indices=np.where(lev_count==len(camchem_levels),-1,np.argmax(above_target,axis=0)) 

        # Select indices of 2 closest neighbouring levels
        vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target
        vertical_indices[:,first_negative_indices==0] = 0 # If the sample lies below the lowest model level. Set it to the lowest model level !!Note: this also asigns a 0 if the pressure is nan, but for our case, these points are filtered out later anyways
        vertical_indices[:,first_negative_indices==-1] = len(camchem_levels)-1 # If the sample lies above the highest model level. Set it to the highest model level

        # Select pressure values of 2 closest neighbouring levels
        pk_low = np.take_along_axis((camchem_levels), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
        pk_up = np.take_along_axis((camchem_levels), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

        # Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
        alpha_low = (np.log(target_pressures/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
        alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

        # Correct alpha for levels lower or higher than the model levels
        alpha_complete[:,first_negative_indices==0] = 0.5 # If sample lies below lowest model level, give all weight to the lowest model level
        alpha_complete[:,first_negative_indices==-1] = 0.5 # If sample lies above the highest model level, give all weight to the highest model level

        # Flip camchem species to match TM5 vertical levels and drop the time dimension
        camchem_subset= np.flip(ds_camchem[species].values[0], axis=0) 

        # Perform the interpolation using the determined weights
        species_interp = np.take_along_axis(camchem_subset, vertical_indices[1], axis=0) * alpha_complete[0] + \
                        np.take_along_axis(camchem_subset, vertical_indices[0], axis=0) * alpha_complete[1]

        # Add the interpolated camchem species to the TROPOMI dataset
        ds_interpolated[species] = (('pres', 'ncells'), species_interp)

    return ds_interpolated


In [321]:

# Define the paths to the camchem files
base_path = '/scratch/snx3000/amols/data/camchem_data'
out_path = '/scratch/snx3000/amols/data/camchem_data/pres_interp'

# Define the list of months, days, and hours
months = ['02','07']  # Add more months as needed
# months = ['03', '08']  # Add more months as needed
# days = list(range(1, 12))  # Days 13-26
days = list(range(12, 29))  # Days 13-26
# hours = ['00', '06', '12', '18']  # Add more hours as needed
hours = ['12', '00']  # Add more hours as needed

# Iterate over months, days, and hours
for month in months:
    for day in days:
        for hour in hours:
            # Load the camchem dataset for the specified month, day, and hour
            camchem_file = os.path.join(base_path, f'camchem_2019{month}{day:02d}{hour}_icon.nc')
            ds_camchem = xr.open_dataset(camchem_file)

            # Calculate pressure levels
            ds_camchem['pres'] = ds_camchem['hyam']*ds_camchem['P0'] + ds_camchem['hybm']*ds_camchem['PS'] # level on which the variables are defined

            # Extracting the selected variables from ds_camchem
            selected_variables = ['clon', 'clat', 'clon_bnds', 'clat_bnds']

            # Creating a new dataset with selected variables
            ds_new = xr.Dataset({var: ds_camchem[var] for var in selected_variables})

            # Creating a new DataArray for the 'pres' variable using target_pressures
            pres_values = np.array([100000, 95000, 90000, 80000, 60000, 40000])
            ncells_values = ds_camchem['ncells'].values

            # Repeat the 'pres' values for each 'ncells'
            pres_dataarray = xr.DataArray(np.tile(pres_values, (len(ncells_values), 1)), dims=('ncells', 'pres'),
                                          coords={'ncells': ncells_values, 'pres': pres_values})

            # Add the value of pres_sfc to pres_dataarray
            pres_sfc_value = ds_camchem['PS'][0].values
            
            # Concatenate pres_sfc_value with pres_dataarray along the second axis
            pres_values_with_sfc = np.concatenate((np.expand_dims(pres_sfc_value, axis=1), pres_dataarray), axis=1)

            # Update pres_dataarray with the concatenated values
            pres_dataarray = xr.DataArray(pres_values_with_sfc, dims=('ncells', 'pres'),
                                        coords={'ncells': ncells_values, 'pres': pres_values_with_sfc[0]})

            # Adding the 'pres' variable to the new dataset
            ds_new['pres'] = pres_dataarray

            # Transpose the 'pres' variable to reorder the coordinates
            ds_new['pres'] = ds_new['pres'].transpose('pres', 'ncells')

            # Perform vertical interpolation for the specified species
            species_list = ['O3', 'NO2', 'NO', 'HNO3', 'N2O5', 'ALKNIT', 'HO2NO2', 'HONITR', 'ISOPNITA', 'ISOPNITB', 'MPAN', 'NO3', 
                            'NOA', 'ONITR', 'PAN', 'PBZNIT', 'TERPNIT', 'OH', 'CH3CHO', 'CO', 'T']
            
            ds_interpolated = vertical_interp_camchem(ds_camchem, ds_new, species_list)

            ds_interpolated['NOx'] = ds_interpolated['NO2'] + ds_interpolated['NO']
            ds_interpolated['NOz'] = ds_interpolated['HNO3'] + ds_interpolated['N2O5'] + ds_interpolated['ALKNIT'] + \
                                    ds_interpolated['HO2NO2'] + ds_interpolated['HONITR'] + ds_interpolated['ISOPNITA'] + \
                                    ds_interpolated['ISOPNITB'] + ds_interpolated['MPAN'] + ds_interpolated['NO3'] + \
                                    ds_interpolated['NOA'] + ds_interpolated['ONITR'] + ds_interpolated['PAN'] + \
                                    ds_interpolated['PBZNIT'] + ds_interpolated['TERPNIT']
            ds_interpolated['NOy'] = ds_interpolated['NOx'] + ds_interpolated['NOz']
            ds_interpolated['NOx_NOy'] = ds_interpolated['NOx']/ds_interpolated['NOy']
            ds_interpolated['NOx_NO2'] = ds_interpolated['NOx']/ds_interpolated['NO2']
            ds_interpolated['temp'] = ds_interpolated['T']

            # Create a directory to store the interpolated datasets if it doesn't exist
            if not os.path.exists(out_path):
                os.makedirs(out_path)

            # Define the output filename
            out_filename = f'interp_camchem_2019{month}{day:02d}T{hour}.nc'
            out_file = os.path.join(out_path, out_filename)

            if os.path.exists(out_file):
                # Delete the file
                os.remove(out_file)

            # Save the interpolated dataset to a NetCDF file
            ds_interpolated.to_netcdf(out_file)

            print(f'Interpolated dataset saved: {out_file}')


Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190212T12.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190212T00.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190213T12.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190213T00.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190214T12.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190214T00.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190215T12.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190215T00.nc
Interpolated dataset saved: /scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190216T12.nc
I

In [7]:
ds = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/pres_interp/interp_camchem_20190212T12.nc')
ds.NO2[2,300].values

array(6.53043175e-09)

In [17]:
ds_icon = xr.open_dataset('/scratch/snx3000/amols/data/nudged_ICON_data/icon-art-full-chem_EU_LBC_20190215T120000Z.nc')

ds_camchem = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_2019021512_icon.nc')
ds_camchem_new = xr.open_dataset('/scratch/snx3000/amols/camchem_2019022200_icon.nc')

#### Calculate camchem height with barometric formula

R0 = 8.3144598          # gas constant J/(mol·K)
g = 9.81                # gravity m/s2
M = 0.02896968  	    # molar mass dry air (kg/mol)
p0 = 101325            # Sea level standard atmospheric pressure (Pa)

ds_camchem['pres_ifc'] = ds_camchem['hyai']*ds_camchem['P0'] + ds_camchem['hybi']*ds_camchem['PS'] # Seperating levels
ds_camchem['pres'] = ds_camchem['hyam']*ds_camchem['P0'] + ds_camchem['hybm']*ds_camchem['PS'] # level on which the variables are defined
ds_camchem['pres_ifc'] = ds_camchem['pres_ifc'].transpose('time', 'ilev', 'ncells')
ds_camchem['pres'] = ds_camchem['pres'].transpose('time', 'lev', 'ncells')

# Barometric formula
ds_camchem['z_mc'] = (-(ds_camchem['T'][:,0,:]*R0)/(g*M)*np.log(ds_camchem['pres']/p0))[0]
ds_camchem['z_ifc'] = (-(ds_camchem['T'][:,0,:]*R0)/(g*M)*np.log(ds_camchem['pres_ifc']/p0))[0]


# switch height and ncells coordinates
ds_camchem['z_mc'] = ds_camchem['z_mc'].transpose("lev", "ncells")

# Transpose the coordinates of z_ifc
ds_camchem['z_ifc'] = ds_camchem['z_ifc'].transpose("ilev", "ncells")

#### Calculate camchem height from geopotential height

Rc = 3963000

# Barometric formula
ds_camchem['z_mc_geo'] = ((ds_camchem.Z3*Rc)/(Rc-ds_camchem.Z3))[0]


# switch height and ncells coordinates
ds_camchem['z_mc_geo'] = ds_camchem['z_mc_geo'].transpose("lev", "ncells")



#ds_icon2 = xr.open_dataset('/scratch/snx3000/amols/camchem_2019022200_icon.nc')


In [20]:
np.nanmean(ds_camchem.z_mc[-1]-ds_camchem.z_mc_geo[-1])

-87.72654992866624

In [22]:
np.nanmedian(ds_camchem.z_mc_geo[-1])

155.73131626555207

In [23]:
np.nanmedian(ds_camchem.z_mc[-1])

105.39514533232338

In [4]:
np.nanmedian(ds_icon.z_mc[-1]-ds_icon.z_ifc[-1])

10.0

In [25]:
np.nanmedian(ds_camchem.z_mc[-1]-ds_camchem.z_ifc[-1])

56.396263643086996

In [32]:
np.nanmin(ds_icon.z_mc[-2]-ds_icon.z_ifc[-1]-(ds_camchem.z_mc[-1]-ds_camchem.z_ifc[-1]))

-27.64958091211315

In [59]:
np.nanmedian(ds_icon.z_mc[-1]-ds_camchem.z_mc[-1])

12.064216376147025

In [63]:
np.nanmedian(ds_icon.pres[0,-1]-ds_camchem.pres[0,-1])

777.8793881376623

In [65]:
np.nanmedian(ds_icon.pres[0,-1])

100762.24

In [70]:
ds_icon.z_mc

<xarray.DataArray 'z_mc' (height: 60, ncells: 90268)> Size: 22MB
[5416080 values with dtype=float32]
Coordinates:
    clon     (ncells) float64 722kB ...
    clat     (ncells) float64 722kB ...
  * height   (height) float64 480B 1.0 2.0 3.0 4.0 5.0 ... 57.0 58.0 59.0 60.0
Dimensions without coordinates: ncells
Attributes:
    standard_name:                geometric_height_at_full_level_center
    long_name:                    geometric height at full level center
    units:                        m
    param:                        6.3.0
    CDI_grid_type:                unstructured
    number_of_grid_in_reference:  1

In [40]:
ds_camchem_orig = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_2019022306_icon.nc')
ds_icon.CH3CHO_full


<xarray.DataArray 'CH3CHO_full' (time: 1, height: 60, ncells: 90268)> Size: 22MB
[5416080 values with dtype=float32]
Coordinates:
  * time     (time) float64 8B 2.019e+07
    clon     (ncells) float64 722kB ...
    clat     (ncells) float64 722kB ...
  * height   (height) float64 480B 1.0 2.0 3.0 4.0 5.0 ... 57.0 58.0 59.0 60.0
Dimensions without coordinates: ncells
Attributes:
    standard_name:                CH3CHO_full
    long_name:                    CH3CHO_full_mixing_ratio
    units:                        kg kg-1
    param:                        1.254.0
    CDI_grid_type:                unstructured
    number_of_grid_in_reference:  1

### Find overlapping ICON and camchem surface layers

In [34]:
ds_icon = xr.open_dataset('/scratch/snx3000/amols/data/nudged_ICON_data/icon-art-full-chem_EU_LBC_20190215T120000Z.nc')
ds_camchem = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_2019021512_icon.nc')

ds_camchem['pres_ifc'] = ds_camchem['hyai']*ds_camchem['P0'] + ds_camchem['hybi']*ds_camchem['PS'] # Seperating levels
ds_camchem['pres'] = ds_camchem['hyam']*ds_camchem['P0'] + ds_camchem['hybm']*ds_camchem['PS'] # level on which the variables are defined
ds_camchem['pres_ifc'] = ds_camchem['pres_ifc'].transpose('time', 'ilev', 'ncells')
ds_camchem['pres'] = ds_camchem['pres'].transpose('time', 'lev', 'ncells')

In [35]:
ds_camchem['pres_ifc'][0,-1]

<xarray.DataArray 'pres_ifc' (ncells: 90268)> Size: 722kB
array([98500.78125  , 98336.53125  , 98385.2265625, ..., 96990.984375 ,
       96781.15625  , 97468.7109375])
Coordinates:
    clat     (ncells) float64 722kB ...
    clon     (ncells) float64 722kB ...
    ilev     float64 8B 1e+03
    time     datetime64[ns] 8B 2019-02-15T12:00:00
Dimensions without coordinates: ncells

In [219]:

# Dimension order: level, ncells
icon_pressures = ds_icon['pres_ifc'][0].values
camchem_pressures = ds_camchem['pres_ifc'][0].values

icon_surface_p = ds_icon['pres_ifc'][0,-1].values
camchem_surface_p = ds_camchem['pres_ifc'][0,-1].values

# Pressure difference within first camchem layer
icon_first_layer_p_diff =  icon_surface_p - icon_pressures[-2]
camchem_first_layer_p_diff =  camchem_surface_p - camchem_pressures[-2]

# Find target pressure
target_pressure = icon_surface_p - camchem_first_layer_p_diff

# Finds the first index where model lies below target level (so model pressure is higher than target pressure)
above_target = icon_pressures[:,np.newaxis] >= target_pressure
first_negative_indices = np.argmax(above_target, axis=0)

# Select indices of 2 closest neighbouring levels
vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target

# # Select pressure values of 2 closest neighbouring levels
pk_low = np.take_along_axis((icon_pressures), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
pk_up = np.take_along_axis((icon_pressures), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

# Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
alpha_low = (np.log(target_pressure/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

# Find which icon levels fully lie within the camchem layer
nr_full_layers = 60 - first_negative_indices

# Calculate the maximum number of full_count_levels
max_levels = np.max(nr_full_layers)

# Initialize an empty list to store arrays
full_count_levels_arrays = []
weights_full_levels = []

# Loop through each level
for i in range(0, max_levels):
    # Create a new array based on the condition
    new_array = np.where(nr_full_layers > i, vertical_indices[0] + i, 0)
    new_array2 = np.where(nr_full_layers > i, 1, 0)
    # Append the new array to the list
    full_count_levels_arrays.append(new_array)
    weights_full_levels.append(new_array2)

# Select the species from the icon dataset
icon_subset= ds_icon.NO2_full[0].values

# Initialize NO2_interp with zeros of the same shape as icon_subset to calculate the weighted average
NO2_interp = np.zeros((1, icon_subset.shape[1]))

# Initialize a count array to keep track of the number of non-zero contributing weights
count_array = np.zeros((1, icon_subset.shape[1]))

NO2_interp = np.take_along_axis(icon_subset, vertical_indices[1], axis=0) * alpha_complete[1]

# Loop through each level array and its corresponding weight array
for i in range(0, max_levels):
    # Get the weighted interpolated array for the current level
    weighted_interpolated_array = np.take_along_axis(icon_subset, full_count_levels_arrays[i], axis=0) * weights_full_levels[i]
    
    # Update NO2_interp with the weighted interpolated values
    NO2_interp += weighted_interpolated_array
    
    # Update count_array by adding 1 where weights are non-zero
    count_array += (weights_full_levels[i] != 0)

# Calculate the average by dividing by the count_array
NO2_interp /= count_array + alpha_complete[1]



In [220]:
NO2_interp

array([[3.05058660e-09, 2.78696311e-09, 2.91407386e-09, ...,
        1.92207795e-10, 2.03393709e-10, 2.00237889e-10]])

In [210]:
alpha_complete[0,:,300]

array([0.36981669])

In [213]:
vertical_indices[1,:,300]

array([56])

In [215]:
target_pressure[300]

98019.44519552786

In [218]:
icon_pressures[:,300]

array([ 3063.152 ,  4986.6675,  6452.7764,  7898.5947,  9363.109 ,
       10863.69  , 12431.795 , 14055.336 , 15733.719 , 17499.865 ,
       19366.402 , 21290.834 , 23264.79  , 25271.662 , 27310.203 ,
       29375.193 , 31452.703 , 33532.758 , 35613.348 , 37693.42  ,
       39769.965 , 41839.836 , 43900.613 , 45950.117 , 47986.305 ,
       50007.31  , 52011.535 , 53997.98  , 55965.777 , 57913.652 ,
       59840.27  , 61744.688 , 63626.66  , 65486.05  , 67321.82  ,
       69133.5   , 70920.45  , 72680.125 , 74408.82  , 76103.1   ,
       77760.41  , 79378.65  , 80955.66  , 82489.4   , 83978.055 ,
       85419.88  , 86813.66  , 88157.13  , 89449.75  , 90689.44  ,
       91874.43  , 93001.05  , 94064.34  , 95060.44  , 95984.695 ,
       96832.19  , 97596.445 , 98268.53  , 98835.18  , 99268.88  ,
       99514.54  ], dtype=float32)

In [4]:
ds_icon = xr.open_dataset('/scratch/snx3000/amols/data/nudged_ICON_data/icon-art-full-chem_EU_LBC_20190212T010000Z.nc')
ds_camchem = xr.open_dataset('/scratch/snx3000/amols/data/camchem_data/camchem_2019021200_icon.nc')

In [69]:
species = 'NO2'

In [295]:
###########################################################################
ds_camchem['pres_ifc'] = ds_camchem['hyai']*ds_camchem['P0'] + ds_camchem['hybi']*ds_camchem['PS'] # Seperating levels
ds_camchem['pres'] = ds_camchem['hyam']*ds_camchem['P0'] + ds_camchem['hybm']*ds_camchem['PS'] # level on which the variables are defined
ds_camchem['pres_ifc'] = ds_camchem['pres_ifc'].transpose('time', 'ilev', 'ncells')
ds_camchem['pres'] = ds_camchem['pres'].transpose('time', 'lev', 'ncells')

# Perform interpolation for surface levels

# Dimension order: level, ncells
icon_depths = ds_icon['z_ifc'][:-1].values-ds_icon['z_ifc'][1:].values
icon_pressures = ds_icon['pres_ifc'][0].values
camchem_pressures = ds_camchem['pres_ifc'][0].values

icon_surface_p = ds_icon['pres_ifc'][0,-1].values
camchem_surface_p = ds_camchem['pres_ifc'][0,-1].values

# Pressure difference within first camchem layer
camchem_first_layer_p_diff =  camchem_surface_p - camchem_pressures[-2]

# Find target pressure
target_pressure = icon_surface_p - camchem_first_layer_p_diff

# Finds the first index where model lies below target level (so model pressure is higher than target pressure)
above_target = icon_pressures[:,np.newaxis] >= target_pressure
first_negative_indices = np.argmax(above_target, axis=0)

# Select indices of 2 closest neighbouring levels
vertical_indices = np.stack([first_negative_indices, first_negative_indices-1], axis=0) # Second index thus lies /above/ the target

# # Select pressure values of 2 closest neighbouring levels
pk_low = np.take_along_axis((icon_pressures), vertical_indices[1], axis=0) #pk, this is the pressure of the lower level (higher pressure)
pk_up = np.take_along_axis((icon_pressures), vertical_indices[0], axis=0) #pk-1, this is the pressure of the higher level (lower pressure) 

# Compute the weights of both pressure levels (using the fact that only ln(P) is linear with height)
alpha_low = (np.log(target_pressure/pk_up))/(np.log(pk_low/pk_up)) # Weights for the lower closest neighbour
alpha_complete = np.stack([alpha_low, 1-alpha_low], axis=0) # These are now the complete weights for the vertical_indices

# Find which icon levels fully lie within the camchem layer
nr_full_layers = 60 - first_negative_indices

# Calculate the maximum number of full_count_levels
max_levels = np.max(nr_full_layers)

# Initialize an empty list to store arrays
full_count_levels_arrays = []
weights_full_levels = []
depth_weights = []

# Loop through each level
for i in range(0, max_levels+1):
    # Create a new array based on the condition
    new_array = np.where(nr_full_layers > i, vertical_indices[0] + i, 0)
    new_array2 = np.where(nr_full_layers > i, 1, 0)
    # Append the new array to the list
    full_count_levels_arrays.append(new_array)
    weights_full_levels.append(new_array2)

# Select the species from the icon dataset
icon_subset = ds_icon[species + '_full'][0].values

# Initialize ds_interp with zeros of the same shape as icon_subset to calculate the weighted average
ds_interp_surface = np.zeros((1, icon_subset.shape[1]))

# Initialize a count array to keep track of the number of non-zero contributing weights
count_array = np.zeros((1, icon_subset.shape[1]))

ds_interp_surface = np.take_along_axis(icon_subset, vertical_indices[1], axis=0) * alpha_complete[1]

store_arrays = []
store_counts = []
store_depths = []

# Loop through each level array and its corresponding weight array
for i in range(0, max_levels):
    # Get the weighted interpolated array for the current level
    weighted_interpolated_array = np.take_along_axis(icon_subset, full_count_levels_arrays[i], axis=0) * weights_full_levels[i]
    depths_array = np.take_along_axis(icon_depths, full_count_levels_arrays[i], axis=0) * weights_full_levels[i]

    # Update ds_interp with the weighted interpolated values
    # ds_interp_surface += weighted_interpolated_array
    
    # Update count_array by adding 1 where weights are non-zero
    count_array += (weights_full_levels[i] != 0)

    store_arrays.append(weighted_interpolated_array)
    store_counts.append(weights_full_levels[i])
    store_depths.append(depths_array)

# # Calculate the average by dividing by the count_array
# ds_interp_surface /= count_array + alpha_complete[1]

# Multiply store_arrays by store_depths element-wise
weighted_arrays = np.array(store_arrays) * np.array(store_depths) # weigh concentration by depth
# Sum along the first axis to get a single array
summed_array = np.sum(weighted_arrays, axis=0) # sum over all full layers

# Weigh the arrat 
half_layer_depth_array = np.take_along_axis(icon_depths, full_count_levels_arrays[0]-1, axis=0) # depth of the icon layer that party falls within the camchem layer
weighted_array_half_levels = ds_interp_surface * half_layer_depth_array*alpha_complete[0] # weigh concentration by depth

ds_interp_surface =  summed_array + weighted_array_half_levels

ds_interp_surface /= np.sum(store_depths, axis=0) + half_layer_depth_array*alpha_complete[0]


In [296]:
ds_interp_surface

array([[1.81739412e-09, 2.06237336e-09, 1.87444320e-09, ...,
        9.37341783e-11, 1.04547238e-10, 7.80076108e-11]])